# Imports,  Settings and Paths

In [1]:
comment = 'no_batchnorm_096_3'

In [19]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import torch
import torch.utils.data as data_utils

In [3]:
from Code.Models.c7o2h10_model import DeepPotential, normalize, backtransform
from Code.Models.api import Network
from Code.Models.nn_extentions import AbcExponentialLR

In [4]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
eval_path = './evaluation/c7o2h10'

In [5]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

use cuda:   True


# Read the Data

In [6]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [7]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1

# Train Model

## Instanciate and create Dataloaders

In [8]:
model = DeepPotential(use_cuda=use_cuda,
                      eval_path=eval_path,
                      comment=comment)
network = Network(model)

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU1 NVS 310 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [9]:
network.create_dataloaders(X_npy, Y_npy)

## Do the learning

In [ ]:
network.fit(350)

epoch: 1	progress: 0.0	time estimate: 281.13	train loss: 0.00369	test loss: 0.001834
epoch: 2	progress: 0.01	time estimate: 286.4	train loss: 0.00152	test loss: 0.001394
epoch: 3	progress: 0.01	time estimate: 284.71	train loss: 0.001395	test loss: 0.001266
epoch: 4	progress: 0.01	time estimate: 290.81	train loss: 0.001284	test loss: 0.001188
epoch: 5	progress: 0.01	time estimate: 283.89	train loss: 0.001199	test loss: 0.001702
epoch: 6	progress: 0.02	time estimate: 273.57	train loss: 0.001148	test loss: 0.001159
epoch: 7	progress: 0.02	time estimate: 270.63	train loss: 0.001087	test loss: 0.00102
epoch: 8	progress: 0.02	time estimate: 268.89	train loss: 0.001056	test loss: 0.000991
epoch: 9	progress: 0.03	time estimate: 268.7	train loss: 0.001019	test loss: 0.000988
epoch: 10	progress: 0.03	time estimate: 270.64	train loss: 0.000987	test loss: 0.00104
epoch: 11	progress: 0.03	time estimate: 267.26	train loss: 0.000964	test loss: 0.000924
epoch: 12	progress: 0.03	time estimate: 267.34	t

In [16]:
mae = network.calculate_test_mae()
mae

0.11613484221795595

In [ ]:
network.model.train()
latent, labels = [], []
for x, y in network.test_loader:
    if network.model.use_cuda:
        x = x.cuda()
    pred = network.model(x)
    latent.append(pred.detach().cpu())
    labels.append(y)
latent = torch.cat(latent).data
labels = torch.cat(labels).data
latent = backtransform(latent, network.y_min, network.y_max).squeeze()
labels = backtransform(labels, network.y_min, network.y_max).squeeze()
abs(latent - labels).mean().numpy().item()

In [20]:
network.show_loss_plot()

/home/mi/stefaah94/miniconda3/lib/python3.6/site-packages/matplotlib/figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [22]:
network.test_losses

[0.0018338283355703135,
 0.0013941265390771162,
 0.0012660046944749535,
 0.0011876590809211714,
 0.001702100103271496,
 0.0011585739846192,
 0.0010197210661341061,
 0.0009914807337863241,
 0.0009881184978099641,
 0.0010401033244637713,
 0.0009236517092360747,
 0.0009204263283059184,
 0.0009046287254951793,
 0.0009643610832945044,
 0.0008893155767784528,
 0.0008484323987461139,
 0.0008342010417922112,
 0.0008123710466489397,
 0.0008336759036716133,
 0.0008064939450719794,
 0.000789872525883421,
 0.0008005098684657103,
 0.0007642604056487983,
 0.0007497807669346105,
 0.0007885259087687605,
 0.0007596226030032728,
 0.0009846253572493907,
 0.0007337735676942469,
 0.0007052968303967013,
 0.0006980309181904866,
 0.0007209450830468897,
 0.0007007978517495672,
 0.0008072582140892017,
 0.0006747027115083328,
 0.0007437636275757219,
 0.000746735755835384,
 0.000732845371761718,
 0.0006878047683206287,
 0.0006573638330889863,
 0.0006661689063140115,
 0.0007658668999888237,
 0.0006393768362305016,

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
network.model.children()